In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [19]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 'replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL',
 '(EOF or read error, treating as "[N]one" ...)']

In [12]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [13]:
# Vectorize the data.
eng_texts = []
fr_texts = []
eng_characters = set()
fr_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
lines[10]

'Run!\tFilez !\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #2077455 (sacredceltic)'

In [14]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    eng_text, fr_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    fr_text = "\t" + fr_text + "\n"
    eng_texts.append(eng_text)
    fr_texts.append(fr_text)
    
    for char in eng_text:
        if char not in eng_characters:
            eng_characters.add(char)
            
    for char in fr_text:
        if char not in fr_characters:
            fr_characters.add(char)

In [15]:
eng_characters = sorted(list(eng_characters))
fr_characters = sorted(list(fr_characters))

num_encoder_tokens = len(eng_characters)
num_decoder_tokens = len(fr_characters)

max_encoder_seq_length = max([len(txt) for txt in eng_texts])
max_decoder_seq_length = max([len(txt) for txt in fr_texts])

print("Number of samples:", len(eng_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [18]:
input_token_index = dict([(char, i) for i, char in enumerate(eng_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(fr_characters)])

encoder_input_data = np.zeros(
    (len(eng_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(eng_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(eng_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (eng_text, fr_text) in enumerate(zip(eng_texts, fr_texts)):
    
    for t, char in enumerate(eng_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
        
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    
    for t, char in enumerate(fr_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [20]:
decoder_target_data.shape

(10000, 59, 93)

In [21]:
decoder_input_data.shape

(10000, 59, 93)

In [28]:
eng_text = eng_texts[0]
fr_text = fr_texts[0]

In [17]:
eng_text

'Go.'

In [18]:
fr_text

'\tVa !\n'

In [25]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [26]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100
13/13 [==============================] - 3s 81ms/step - loss: 2.3566 - accuracy: 0.5837 - val_loss: 1.8184 - val_accuracy: 0.6077
Epoch 2/100
13/13 [==============================] - 1s 50ms/step - loss: 1.6639 - accuracy: 0.6348 - val_loss: 1.6969 - val_accuracy: 0.6077
Epoch 3/100
13/13 [==============================] - 1s 51ms/step - loss: 1.6026 - accuracy: 0.6358 - val_loss: 1.6538 - val_accuracy: 0.6056
Epoch 4/100
13/13 [==============================] - 1s 53ms/step - loss: 1.5504 - accuracy: 0.6397 - val_loss: 1.6903 - val_accuracy: 0.6112
Epoch 5/100
13/13 [==============================] - 1s 54ms/step - loss: 1.5409 - accuracy: 0.6391 - val_loss: 1.5970 - val_accuracy: 0.6141
Epoch 6/100
13/13 [==============================] - 1s 50ms/step - loss: 1.4813 - accuracy: 0.6399 - val_loss: 1.5519 - val_accuracy: 0.6115
Epoch 7/100
13/13 [==============================] - 1s 49ms/step - loss: 1.4379 - accuracy: 0.6427 - val_loss: 1.6551 - val_accuracy: 0.6139
Epoch 

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [10]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [11]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Continue !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Continue !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Continue !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Go.
Decoded sentence: Continue !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Hi.
Decoded sentence: Falut !

1/1 [==============================] - 0s 13ms/step
-
Input sentence: Hi.
Decoded sentence: Falut !

1/1 [==============================] - 0s 14ms/step
-
Input sentence: Run!
Decoded sentence: Cours !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run!
Decoded sentence: Cours !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run!
Decoded sentence: Cours !

1/1 [==============================] - 0s 12ms/step
-
Input sentence: Run!
D

In [12]:
encoder_input_data[seq_index : seq_index + 1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtype=float32)